In [ ]:
import os
import numpy as np
import pandas as pd
from scipy.io import loadmat

def safe_get(d, key):
    """Safely get a field from MATLAB struct or return empty array."""
    return d[key][0].flatten() if key in d.dtype.names else np.array([])

def extract_battery_data(mat_path, save_dir, group_name, metadata_records):
    file_name = os.path.basename(mat_path).split('.')[0]
    print(f"\n🔍 Reading {file_name} from group '{group_name}'...")

    data = loadmat(mat_path)
    battery = data[file_name][0, 0]
    cycles = battery['cycle'][0]

    charge_records, discharge_records, impedance_records = [], [], []

    for i, cycle in enumerate(cycles, 1):
        cycle_type = str(cycle['type'][0]).strip().lower()
        data_struct = cycle['data'][0, 0]
        ambient_temp = float(cycle['ambient_temperature'][0][0]) if 'ambient_temperature' in cycle.dtype.names else np.nan

        print(f"Cycle {i} → {cycle_type} | Ambient Temp: {ambient_temp}")

        # ---------- CHARGE ----------
        if cycle_type == 'charge':
            df = pd.DataFrame({
                'Group_Name': group_name,
                'Battery_ID': file_name,
                'Cycle_No': i,
                'Ambient_Temperature': ambient_temp,
                'Time': safe_get(data_struct, 'Time'),
                'Voltage_measured': safe_get(data_struct, 'Voltage_measured'),
                'Current_measured': safe_get(data_struct, 'Current_measured'),
                'Temperature_measured': safe_get(data_struct, 'Temperature_measured'),
                'Current_charge': safe_get(data_struct, 'Current_charge'),
                'Voltage_charge': safe_get(data_struct, 'Voltage_charge')
            })
            charge_records.append(df)

        # ---------- DISCHARGE ----------
        elif cycle_type == 'discharge':
            # Safely extract capacity
            try:
                if 'Capacity' in data_struct.dtype.names and len(data_struct['Capacity']) > 0:
                    cap_value = float(data_struct['Capacity'][0][0])
                else:
                    cap_value = np.nan
            except Exception:
                cap_value = np.nan

            df = pd.DataFrame({
                'Group_Name': group_name,
                'Battery_ID': file_name,
                'Cycle_No': i,
                'Ambient_Temperature': ambient_temp,
                'Time': safe_get(data_struct, 'Time'),
                'Voltage_measured': safe_get(data_struct, 'Voltage_measured'),
                'Current_measured': safe_get(data_struct, 'Current_measured'),
                'Temperature_measured': safe_get(data_struct, 'Temperature_measured'),
                'Capacity': cap_value
            })
            discharge_records.append(df)

        # ---------- IMPEDANCE ----------
        elif cycle_type == 'impedance':
            fields = ['Sense_current', 'Battery_current', 'Current_ratio',
                      'Battery_impedance', 'Rectified_impedance', 'Re', 'Rct']
            arrays = {f: safe_get(data_struct, f) for f in fields}
            valid_arrays = [a for a in arrays.values() if len(a) > 0]
            if not valid_arrays:
                print(f"⚠️ Skipping impedance cycle {i} — no valid data found.")
                continue
            min_len = min(len(a) for a in valid_arrays)
            df = pd.DataFrame({
                'Group_Name': group_name,
                'Battery_ID': file_name,
                'Cycle_No': i,
                'Ambient_Temperature': ambient_temp,
                **{f: arrays[f][:min_len] for f in fields if len(arrays[f]) > 0}
            })
            impedance_records.append(df)

    # ✅ Save in organized folder structure
    group_save_path = os.path.join(save_dir, group_name)
    os.makedirs(group_save_path, exist_ok=True)

    if charge_records:
        path = os.path.join(group_save_path, f'{file_name}_charge.csv')
        pd.concat(charge_records, ignore_index=True).to_csv(path, index=False)
        metadata_records.append([file_name, group_name, path, 'charge'])
        print(f"✅ Saved {file_name}_charge.csv")

    if discharge_records:
        path = os.path.join(group_save_path, f'{file_name}_discharge.csv')
        pd.concat(discharge_records, ignore_index=True).to_csv(path, index=False)
        metadata_records.append([file_name, group_name, path, 'discharge'])
        print(f"✅ Saved {file_name}_discharge.csv")

    if impedance_records:
        path = os.path.join(group_save_path, f'{file_name}_impedance.csv')
        pd.concat(impedance_records, ignore_index=True).to_csv(path, index=False)
        metadata_records.append([file_name, group_name, path, 'impedance'])
        print(f"✅ Saved {file_name}_impedance.csv")


def process_all_battery_groups(base_dataset_folder, save_dir):
    metadata_records = []
    for root, dirs, files in os.walk(base_dataset_folder):
        mat_files = [f for f in files if f.endswith('.mat')]
        if mat_files:
            group_name = os.path.basename(root)
            print(f"\n📂 Found group: {group_name} ({len(mat_files)} files)")
            for mat_file in mat_files:
                mat_path = os.path.join(root, mat_file)
                extract_battery_data(mat_path, save_dir, group_name, metadata_records)

    meta_df = pd.DataFrame(metadata_records, columns=['Battery_ID', 'Group_Name', 'File_Path', 'Type'])
    meta_csv_path = os.path.join(save_dir, 'Master_Metadata.csv')
    meta_df.to_csv(meta_csv_path, index=False)
    print(f"\n✅ Master metadata saved at:\n{meta_csv_path}")
    print(f"Total files processed: {len(metadata_records)}")


# 🔧 Paths
base_dataset_folder = '/content/drive/MyDrive/4 month project/DataSet'  # folder with group subfolders
save_folder = '/content/drive/MyDrive/4 month project/DataSet/Extracted Files'  # output folder

process_all_battery_groups(base_dataset_folder, save_folder)


Streaming output truncated to the last 5000 lines.
Cycle 179 → discharge | Ambient Temp: 4.0
Cycle 180 → charge | Ambient Temp: 4.0
Cycle 181 → discharge | Ambient Temp: 4.0
Cycle 182 → impedance | Ambient Temp: 24.0
Cycle 183 → charge | Ambient Temp: 4.0
Cycle 184 → impedance | Ambient Temp: 24.0
✅ Saved B0047_charge.csv
✅ Saved B0047_discharge.csv
✅ Saved B0047_impedance.csv

🔍 Reading B0048 from group 'BatteryAgingARC_45_46_47_48'...
Cycle 1 → discharge | Ambient Temp: 4.0
Cycle 2 → impedance | Ambient Temp: 24.0
Cycle 3 → charge | Ambient Temp: 4.0
Cycle 4 → impedance | Ambient Temp: 24.0
Cycle 5 → discharge | Ambient Temp: 4.0
Cycle 6 → charge | Ambient Temp: 4.0
Cycle 7 → discharge | Ambient Temp: 4.0
Cycle 8 → charge | Ambient Temp: 4.0
Cycle 9 → discharge | Ambient Temp: 4.0
Cycle 10 → charge | Ambient Temp: 4.0
Cycle 11 → discharge | Ambient Temp: 4.0
Cycle 12 → charge | Ambient Temp: 4.0
Cycle 13 → discharge | Ambient Temp: 4.0
Cycle 14 → impedance | Ambient Temp: 24.0
Cycle 

In [4]:
import os
import numpy as np
from scipy.io import loadmat

# Define the path to a sample .mat file
# You can choose any .mat file from your dataset, for example:
sample_mat_file_path = '/content/drive/MyDrive/4 month project/DataSet/Mat Files/BatteryAgingARC_25-44/B0025.mat'

print(f"Inspecting: {sample_mat_file_path}")

# Load the .mat file
mat_data = loadmat(sample_mat_file_path)

# The loadmat function returns a dictionary. Let's see its keys.
print("\nKeys in the .mat file (dictionary keys):")
for key in mat_data.keys():
    print(f"- {key}")

# Usually, the actual data is under a key that matches the file name (without extension)
# or a key like '__header__', '__version__', '__globals__'.
# Let's try to access the main battery data structure if it exists.
file_name_without_ext = os.path.basename(sample_mat_file_path).split('.')[0]
if file_name_without_ext in mat_data:
    battery_data = mat_data[file_name_without_ext]
    print(f"\nStructure of '{file_name_without_ext}':")
    # MATLAB structures are often represented as NumPy structured arrays or object arrays
    # We can print the data type names to see the top-level fields
    if isinstance(battery_data, np.ndarray) and battery_data.dtype.names:
        print(f"Top-level fields: {battery_data.dtype.names}")
        # To delve deeper, you'd typically access elements like this:
        # cycle_data = battery_data[0, 0]['cycle'][0]
        # And then inspect `cycle_data.dtype.names` for cycle-specific fields.
        print("To explore further, you would access nested fields. For example:")
        print("  battery_data[0, 0]['cycle'][0].dtype.names")
    else:
        print(f"Type: {type(battery_data)}")
        print("This might require more specific indexing based on its structure.")
else:
    print(f"\nCould not find a key matching the file name '{file_name_without_ext}'.")
    print("You might need to manually inspect 'mat_data' for the relevant key.")


Inspecting: /content/drive/MyDrive/4 month project/DataSet/Mat Files/BatteryAgingARC_25-44/B0025.mat

Keys in the .mat file (dictionary keys):
- __header__
- __version__
- __globals__
- B0025

Structure of 'B0025':
Top-level fields: ('cycle',)
To explore further, you would access nested fields. For example:
  battery_data[0, 0]['cycle'][0].dtype.names


In [5]:
cycle_data = battery_data[0, 0]['cycle'][0]
print(f"Fields inside 'cycle': {cycle_data.dtype.names}")

Fields inside 'cycle': ('type', 'ambient_temperature', 'time', 'data')


In [9]:
first_cycle = cycle_data[3]

print(f"Content of 'type': {first_cycle['type']}")
print(f"Content of 'ambient_temperature': {first_cycle['ambient_temperature']}")

# For 'time', it's likely an array, so show its type and first few elements/shape
print(f"Content of 'time' (Type: {type(first_cycle['time'])}, Shape: {first_cycle['time'].shape}):\n{first_cycle['time'][:5]}...")

# For 'data', it's often a nested structure, so show its type and inner fields if available
print(f"Content of 'data' (Type: {type(first_cycle['data'])}):")
if isinstance(first_cycle['data'], np.ndarray) and first_cycle['data'].dtype.names:
    print(f"  Inner fields of 'data': {first_cycle['data'].dtype.names}")
else:
    print(f"  {first_cycle['data']}")

Content of 'type': ['discharge']
Content of 'ambient_temperature': [[24]]
Content of 'time' (Type: <class 'numpy.ndarray'>, Shape: (1, 6)):
[[2.0090e+03 2.0000e+00 1.3000e+01 2.3000e+01 1.2000e+01 2.8078e+01]]...
Content of 'data' (Type: <class 'numpy.ndarray'>):
  Inner fields of 'data': ('Voltage_measured', 'Current_measured', 'Temperature_measured', 'Current_load', 'Voltage_load', 'Time', 'Capacity')
